In [1]:
import pandas as pd

data = pd.read_csv('rainfall-riverheight-history-Rathnapura.csv')

data.head()

Date      station  river_hight  rainfall
0  6/14/2024    Putupaula         0.75       1.4
1  6/14/2024     Ellagawa         5.25       0.0
2  6/14/2024   Rathnapura         1.38       0.0
3  6/14/2024       Magura         1.75       0.0
4  6/14/2024  Kalawellawa         2.47       0.0

In [2]:
rathnapura_data = data[data['station'] == 'Magura']

# Convert 'Date' to datetime format
rathnapura_data['Date'] = pd.to_datetime(rathnapura_data['Date'])

# Convert 'Date' to datetime format
rathnapura_data['Date'] = pd.to_datetime(rathnapura_data['Date'])

# Check the processed data and look for missing values
rathnapura_data.info(), rathnapura_data.head()

# Filter the data for the "Rathnapura" station and sort by date
rathnapura_data = data[data['station'] == 'Magura'].sort_values('Date')


#rename river height column as 'river_height'
rathnapura_data.rename(columns={'river_hight': 'river_height'}, inplace=True)


rathnapura_data.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 3 to 1808
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         362 non-null    datetime64[ns]
 1   station      362 non-null    object        
 2   river_hight  362 non-null    float64       
 3   rainfall     358 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 14.1+ KB


C:\Users\Madusanka(MadusankaB\AppData\Local\Temp\ipykernel_17036\3940579642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rathnapura_data['Date'] = pd.to_datetime(rathnapura_data['Date'])
C:\Users\Madusanka(MadusankaB\AppData\Local\Temp\ipykernel_17036\3940579642.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rathnapura_data['Date'] = pd.to_datetime(rathnapura_data['Date'])


Date station  river_height  rainfall
293    1/1/2024  Magura          1.80      20.9
1373  1/10/2022  Magura          4.88      24.3
248   1/10/2024  Magura          1.61       0.0
243   1/11/2024  Magura          1.58       1.2
238   1/12/2024  Magura          1.41       0.0

In [3]:
# remove outliers
rathnapura_data = rathnapura_data[rathnapura_data['rainfall'] < 200]

# add rolling mean to missing values
rathnapura_data['rainfall'] = rathnapura_data['rainfall'].fillna(rathnapura_data['rainfall'].rolling(30, min_periods=1).mean())

# #fill missing values with the mean of the monthly rainfall for each month
# rathnapura_data['rainfall'] = rathnapura_data.groupby(rathnapura_data['Date'].dt.month)['rainfall'].transform(lambda x: x.fillna(x.mean()))

# plot the data
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(rathnapura_data['Date'], rathnapura_data['rainfall'], label='Rainfall')
plt.title('Rainfall in Magura')
plt.xlabel('Date')
plt.ylabel('Rainfall (mm)')
plt.legend()
plt.show()

In [4]:
#remove outliers
rathnapura_data = rathnapura_data[rathnapura_data['river_height'] < 10]

#add rolling mean to missing values
rathnapura_data['river_height'] = rathnapura_data['river_height'].fillna(rathnapura_data['river_height'].rolling(30, min_periods=1).mean())


#plot river height
plt.figure(figsize=(12, 6))
plt.plot(rathnapura_data['Date'], rathnapura_data['river_height'], label='River Height')
plt.title('River Height in Magura')
plt.xlabel('Date')
plt.ylabel('River Height (m)')
plt.legend()
plt.show()

In [5]:
# Creating lagged features for river height and rainfall
for lag in range(1, 4):
    rathnapura_data[f'river_height_lag_{lag}'] = rathnapura_data['river_height'].shift(lag)
    rathnapura_data[f'rainfall_lag_{lag}'] = rathnapura_data['rainfall'].shift(lag)

# Dropping rows with missing values that result from lagging
rathnapura_data.dropna(inplace=True)

# Splitting the data into train and test sets
# Let's use the last 20% of the data as test set
split_index = int(len(rathnapura_data) * 0.8)
train_data = rathnapura_data.iloc[:split_index]
test_data = rathnapura_data.iloc[split_index:]

# Show the structure of the dataset with new features and the split
train_data.info(), test_data.info(), train_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 283 entries, 243 to 1638
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                283 non-null    object 
 1   station             283 non-null    object 
 2   river_height        283 non-null    float64
 3   rainfall            283 non-null    float64
 4   river_height_lag_1  283 non-null    float64
 5   rainfall_lag_1      283 non-null    float64
 6   river_height_lag_2  283 non-null    float64
 7   rainfall_lag_2      283 non-null    float64
 8   river_height_lag_3  283 non-null    float64
 9   rainfall_lag_3      283 non-null    float64
dtypes: float64(8), object(2)
memory usage: 24.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, 798 to 678
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                71 non-null     object 
 1   sta

(None,
 None,
            Date station  river_height  rainfall  river_height_lag_1  \
 243   1/11/2024  Magura          1.58       1.2                1.61   
 238   1/12/2024  Magura          1.41       0.0                1.58   
 1368  1/13/2022  Magura          1.41       3.0                1.41   
 233   1/13/2024  Magura          1.28       0.0                1.41   
 228   1/14/2024  Magura          1.28       0.0                1.28   
 
       rainfall_lag_1  river_height_lag_2  rainfall_lag_2  river_height_lag_3  \
 243              0.0                4.88            24.3                1.80   
 238              1.2                1.61             0.0                4.88   
 1368             0.0                1.58             1.2                1.61   
 233              3.0                1.41             0.0                1.58   
 228              0.0                1.41             3.0                1.41   
 
       rainfall_lag_3  
 243             20.9  
 238            

In [6]:
from sklearn.preprocessing import StandardScaler

# Features and target variable
features = ['river_height_lag_1', 'rainfall_lag_1', 'river_height_lag_2', 'rainfall_lag_2', 'river_height_lag_3', 'rainfall_lag_3']
target = 'river_height'

# Standardizing the features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_data[features])
test_features = scaler.transform(test_data[features])

train_labels = train_data[target]
test_labels = test_data[target]

# Check the first few rows to see the scaling
train_features[:5], train_labels.head()

(array([[-0.9061815 , -0.66486944,  1.49190295,  0.27288107, -0.76234149,
          0.13904592],
        [-0.92828032, -0.61862552, -0.90742608, -0.66403895,  1.49920413,
          0.27023573],
        [-1.05350693, -0.66486944, -0.92943827, -0.6177713 , -0.90185242,
         -0.66738552],
        [-1.05350693, -0.54925963, -1.05417403, -0.66403895, -0.92388046,
         -0.62108324],
        [-1.14926845, -0.66486944, -1.05417403, -0.54836981, -1.04870603,
         -0.66738552]]),
 243     1.58
 238     1.41
 1368    1.41
 233     1.28
 228     1.28
 Name: river_height, dtype: float64)

In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Assuming 'train_features' and 'train_labels' are your training datasets
# Reshape features for LSTM input (samples, time steps, features)
train_features_reshaped = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))

# Model definition
model = Sequential([
    # Bidirectional LSTM layer
    Bidirectional(LSTM(200, return_sequences=True, input_shape=(train_features_reshaped.shape[1], train_features_reshaped.shape[2]))),
    Dropout(0.2),  # Dropout for regularization
    LSTM(100),      # Second LSTM layer
    Dropout(0.2),  # Another Dropout layer
    Dense(10, activation='relu', kernel_regularizer=l2(0.01)),  # Dense layer with L2 regularization
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mean_absolute_error'])



# Model training
history = model.fit(
    train_features_reshaped,
    train_labels,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    # callbacks=[early_stopping, model_checkpoint],
    verbose=2
)


# Model evaluation (assuming 'test_features' and 'test_labels' are your test datasets)
test_features_reshaped = test_features.reshape((test_features.shape[0], 1, test_features.shape[1]))
test_loss, test_mae = model.evaluate(test_features_reshaped, test_labels, verbose=2)

print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

# Optionally plot the history for loss and validation loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()


Epoch 1/100


C:\Users\Madusanka(MadusankaB\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 - 12s - 783ms/step - loss: 9.0073 - mean_absolute_error: 2.6909 - val_loss: 14.2274 - val_mean_absolute_error: 3.4207
Epoch 2/100
15/15 - 0s - 20ms/step - loss: 8.9202 - mean_absolute_error: 2.6768 - val_loss: 14.0764 - val_mean_absolute_error: 3.4011
Epoch 3/100
15/15 - 0s - 22ms/step - loss: 8.8484 - mean_absolute_error: 2.6652 - val_loss: 13.9159 - val_mean_absolute_error: 3.3803
Epoch 4/100
15/15 - 0s - 20ms/step - loss: 8.7581 - mean_absolute_error: 2.6508 - val_loss: 13.7209 - val_mean_absolute_error: 3.3550
Epoch 5/100
15/15 - 0s - 18ms/step - loss: 8.6549 - mean_absolute_error: 2.6347 - val_loss: 13.5230 - val_mean_absolute_error: 3.3288
Epoch 6/100
15/15 - 0s - 15ms/step - loss: 8.5504 - mean_absolute_error: 2.6173 - val_loss: 13.2847 - val_mean_absolute_error: 3.2971
Epoch 7/100
15/15 - 0s - 14ms/step - loss: 8.4242 - mean_absolute_error: 2.5968 - val_loss: 13.0183 - val_mean_absolute_error: 3.2612
Epoch 8/100
15/15 - 0s - 16ms/step - loss: 8.2938 - mean_absolute_error:

In [8]:
# save the model
model.save('river_height_prediction_magura_model.keras')

In [10]:
#load the model
from tensorflow.keras.models import load_model

model = load_model('river_height_prediction_magura_model.keras')

# Predictions
predictions = model.predict(test_features_reshaped)

# Plotting the predictions
plt.figure(figsize=(12, 6))
plt.plot(test_data['Date'], test_data['river_height'], label='Actual')
plt.plot(test_data['Date'], predictions, label='Predicted')
plt.title('River Height Prediction')
plt.xlabel('Date')
plt.ylabel('River Height (m)')
plt.legend()
plt.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 421ms/step


In [11]:
import requests

# URL of the FastAPI endpoint
url = 'http://127.0.0.1:8000/forecast/magura/'

# Data to be sent to the API
data = {
    "current_height": 2.5,
    "current_rainfall": 10.0
}

# Make the POST request and store the response
response = requests.post(url, json=data)

# Print the status code and the response data
print(f"Status Code: {response.status_code}")
print("Response Data:", response.json())

Status Code: 422
Response Data: {'detail': [{'type': 'string_pattern_mismatch', 'loc': ['path', 'location'], 'msg': "String should match pattern '^(rathnapura|ellagawa|putupaula)$'", 'input': 'magura', 'ctx': {'pattern': '^(rathnapura|ellagawa|putupaula)$'}}]}
